## Logical Routing

In [86]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableLambda

# Data model
class RouteQuery(BaseModel):
    """List with all available datasources, the model chose from in order to generate the response"""

    datasource: Literal["python_docs", "js_docs", "csharp_docs"] = Field(
        ...,
        description="""Given the user query, choose which of these datasources python_docs, js_docs, etc is the most useful to answer the question""",
    )

# LLM with function call 
llm = ChatOllama(model="llama3.1", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at identifying which programming language is being given by the user

Programming languages include python, js, among other, determine which programming language suits the best, your response should provide the 
datasource, datasources are the ones provided at the context.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

Now we invoke the LLM to extract the data source based on the predefined schema:


In [87]:
question = """console.log("Hello!")
"""

result = router.invoke({"question": question})

result.datasource

'js_docs'

Here it's important to note that prompt engineering is a crucial step in this process. The quality of the prompt can significantly influence the model's performance and the relevance of the generated output. Once we've extracted the relevant data source, we can pass the value into another function to exectue additional logic as required:

In [88]:
def choose_route(result):
    available_routes ={
        'python_docs':'chain for python_docs',
        'js_docs':'chain for js_docs',
        'csharp_docs':'chain for csharp_docs'
    }
    for key, value in available_routes.items():
        if key == result.datasource.lower():
            return value
        

full_chain = router | RunnableLambda(choose_route)

First the result is lowercased to ensure consistency, in order to do our chain more resilient to the LLM going off script and producing output that doesn't quite conform to the schema we asked for, the next advanced routing technique is **Semantic Routing**. Let's go back to the main [README_file](../README.md/#Semantic_Routing).